# Advection Equation

Here we solve the advection equation $u_t + a u_x=0$ using as an initial condition a gaussian profile $u(x,t=0)=\exp{[-(x-x_0)^2]}$ with $x_0=5$.

We use a domain $x\in [0,10]$ with periodic boundary conditions.

In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np

!pip install imageio[ffmpeg]

import imageio.v2 as imageio
import glob

In [12]:
method = 'LF'

In [13]:
##create directory where to save images
print(os.getcwd())

os.makedirs('./images_'+method, exist_ok=True)

os.listdir('./')

C:\Users\giuli\OneDrive\Desktop\Astro\numerical


['.git',
 '.ipynb_checkpoints',
 'Advection_Equation.ipynb',
 'FTCS.ipynb',
 'images_FTCS',
 'images_LAXWENDROFF',
 'images_leapfrog',
 'images_LF',
 'l2norm_FTCS.dat',
 'l2norm_LAX.dat',
 'l2norm_LAXWENDROFF.dat',
 'l2norm_LEAPFROG.dat',
 'leapfrog.ipynb',
 'LF.ipynb',
 'LW.ipynb',
 'movie_FTCS.mp4',
 'movie_LAXWENDROFF.mp4',
 'movie_leapfrog.mp4',
 'movie_LF.mp4']

In [14]:
#define the speed a
a=1.0

# Define the domain
L = 10.0     # Domain length
nx = 101    # Number of grid points
dx = L/(nx-1)   # Grid spacing
x = np.linspace(0, L, nx)

##print(x,'\n',dx) #just a check

# Define the time step and the final time
cf = 0.5
dt = cf*dx/a   # Time step
t_final = 20.0

print('nx=',nx)
print('dx=',dx)
print('dt=',dt)
print('Number of iterations=',t_final/dt)


# Define the initial condition
x0=5
u_initial = np.exp(-(x-x0)**2)

# Initialize the solution array
u_current = u_initial.copy()

#we create arrays where we will store the time and the l2norm
l2norm=[]
l2norm.append(np.sqrt(np.sum(u_current**2)/len(u_current)))

time=[]
time.append(0.0)

nx= 101
dx= 0.1
dt= 0.05
Number of iterations= 400.0


In [ ]:
# Initilize time and iteration counter
t = 0.0
i = 0

#save the initial conditions
plt.plot(x, u_current)
plt.title('Time='+str(round(t,2)))
plt.ylim(0,1.1)
plt.savefig('./images_'+method+'/fig_'+str(i).zfill(5)+'.png', dpi=200)
plt.close()

#solve the advection equation
while t < t_final:
    # Compute the new solution using the FTCS method
    # Note: np.roll(u_current, -1) is equivalent to u(j+1) and
    #       np.roll(u_current,  1) is equivalent to u(j-1)
    # using np.roll is equivalent to use periodic boundary conditions
    u_next = 0.5*(np.roll(u_current, -1) + np.roll(u_current, 1)) - a*dt/(2*dx)*(np.roll(u_current, -1) - np.roll(u_current, 1))    
    
    # Update the solution
    u_current = u_next.copy()
    
    
    #advance the time 
    t += dt
    i += 1
    
    #compute the l2 norm and add the time to the time vector
    l2norm.append(np.sqrt(np.sum(u_current**2)/len(u_current)))
    time.append(t)
    
    #plot the current result and save in an image every 10 iterations
    if (i%10==0):
        plt.plot(x, u_current)
        plt.title('Time='+str(round(t,2)))
        plt.ylim(0,1.1)
        plt.savefig('./images_'+method+'/fig_'+str(i).zfill(5)+'.png', dpi=200)
        plt.close()


In [ ]:
# Plot the final solution
plt.plot(x, u_initial, label='Initial')
plt.plot(x, u_current, label='Final')
plt.title('Time='+str(round(t,2)))
plt.ylim(0,1.1)
plt.legend()
plt.show()

In [ ]:
# set the directory where your images are stored
directory = "./images_"+method

# get the list of image files in the directory
files = os.listdir(directory)

print(files, '\n')

# sort the files in alphanumeric order
files=sorted(files)

print(files)




In [ ]:
#get list of images in correct order
images_files = sorted(glob.glob('./images_'+method+'/fig_*.png'))

#use 'ffmpeg' plugin explicitly and specify codec
output_video_path = os.path.abspath('./movie_'+method+'.mp4')

with imageio.get_writer(output_video_path, fps=20, codec='libx264') as writer:
    for file in images_files:
        image = imageio.imread(file)
        writer.append_data(image)

In [ ]:
# don't worry about the code in this cell, it is just to let you 
# display the movies you generated above in Jupyter notebook
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="./movie_FTCS.mp4" type="video/mp4">
</video></div>""")

In [ ]:
plt.plot(time,l2norm)
plt.show()

In [ ]:
#Compare with data stored
l2_saved=np.genfromtxt('./l2norm_LAX.dat', comments='#')
plt.plot(time,l2norm)
plt.plot(l2_saved[:,0], l2_saved[:,1], '+')
plt.show()